In [9]:
import scipy as sp
import xarray as xr

In [10]:
def check_zeta(ds):
    """
    da_stacked containes values of zeta < -1 and their coordinates
    """
    da = ds.zeta.isel(ocean_time=-1)
    da_stacked = da.where(da<-1).stack(x=['eta_rho','xi_rho'])
    da_stacked = da_stacked[da_stacked.notnull()]
    return da_stacked

In [11]:
def check_temp(ds):
    """
    Finds lacations (eta, xi, s) of the points with high temperature
    """
    temp_da = ds.temp.isel(ocean_time=-1)
    temp_da_stacked = temp_da.where(temp_da>12).stack(x=['eta_rho', 'xi_rho', 's_rho'])
    temp_da_stacked = temp_da_stacked[temp_da_stacked.notnull()]
    return temp_da_stacked

In [12]:
def plot_bad_temp(ds, temp_da_point):
    bad_temp_point = temp_da_point
    eta, xi, s = bad_temp_point.eta_rho.values, bad_temp_point.xi_rho.values, bad_temp_point.s_rho.values
    print(f"Xi: {xi}; Eta: {eta}")
    ds.temp.isel(ocean_time=-1, eta_rho=slice(eta-5,eta+5), xi_rho=slice(xi-5,xi+5)).sel(s_rho=s).plot(figsize=(10, 5))

In [15]:
def filter(ds, sigma=1):
    """
    Applies gaussian filter to ds.h.values
    """
    new_values = sp.ndimage.gaussian_filter(ds.h.values, sigma, mode='nearest')
    ds.h.values = new_values
    return ds

In [13]:
def save_to_netcdf(ds, name):
    ds.to_netcdf(path=f'fram_data/norfjords_160m_{name}.nc', format='NETCDF4')

In [14]:
ds = xr.open_dataset('fram_data/norfjords_160m_grid_version2+.nc')